In [1]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

In [56]:
#path = r'C:\Users\adai\Documents\PD Study Data\updated_demo_pd_01\updated_demo_pd\01'
path = r'C:\Users\adai\Downloads\pd_study_v2_0_1004\PD Study v2.0\1004'

files = os.listdir(path)

for entries in files:
    if entries == 'annotations.csv':
        print(os.path.join(path, entries))
        df = pd.read_csv(os.path.join(path, entries))

# removes unnecessary data columns
del df['Timestamp (ms)']
del df['AnnotationId']
del df['AuthorId']

df['Start Timestamp (ms)'] = pd.to_datetime(df['Start Timestamp (ms)'], unit='ms', utc=True).dt.tz_localize('UTC').dt.tz_convert('US/Central')
df['Stop Timestamp (ms)'] = pd.to_datetime(df['Stop Timestamp (ms)'], unit='ms', utc=True).dt.tz_localize('UTC').dt.tz_convert('US/Central')

df.head()

testDay = df[df.EventType == 'Testing Day'].dropna(how='any').Value
print(testDay)

df = df[(df.EventType != 'Testing Day')]
del df['Value']

sorter = df.EventType.unique()
sorterIndex = dict(zip(sorter,range(len(sorter))))

df['EventType_Rank'] = df['EventType'].map(sorterIndex)
df['Cycle'] = df.groupby('EventType')['Start Timestamp (ms)'].rank(ascending=True).astype(int)
repl1 = df['EventType'].str.contains('MDS-UPDRS')
df['Cycle'] = df[repl1]['Cycle'] == 2
df['Cycle'].where(repl2, other=3)
df.sort_values(['EventType', 'EventType_Rank', 'Start Timestamp (ms)'], axis=0)


C:\Users\adai\Downloads\pd_study_v2_0_1004\PD Study v2.0\1004\annotations.csv
2    DAY 1
Name: Value, dtype: object
0      1
3      1
4      1
5      1
6      1
7      1
8      1
9      1
10     1
11     1
12     1
13     1
14     1
15     1
16     1
17     2
18     1
19     1
20     1
21     1
22     1
23     1
24     1
25     1
26     1
27     2
28     2
29     2
30     3
31     4
      ..
75     4
76     4
77     4
78     5
79     5
80     5
81     7
82     6
83     5
84     5
85     5
86     5
87     5
88     5
89     5
90     5
91     6
92     6
93     6
94     8
95     7
96     6
97     6
98     6
99     6
100    6
101    6
102    6
103    6
104    2
Name: Cycle, Length: 103, dtype: int32


,EventType,Start Timestamp (ms),Stop Timestamp (ms),EventType_Rank,Cycle
0,Heart Rate Variability,2017-07-07 08:15:33.880000-05:00,2017-07-07 08:21:33.890000-05:00,0,1
104,Heart Rate Variability,2017-07-07 11:35:53.043000-05:00,2017-07-07 11:41:53.054000-05:00,0,2
50,MDS-UPDRS #10: Kinetic Hand Tremor,2017-07-07 10:02:16.987000-05:00,2017-07-07 10:02:22.242000-05:00,24,1
12,MDS-UPDRS #11: Rest Tremor,2017-07-07 08:35:30.810000-05:00,2017-07-07 08:35:39.710000-05:00,10,1
3,MDS-UPDRS #1: Finger Tapping,2017-07-07 08:32:44.865000-05:00,2017-07-07 08:32:56.490000-05:00,1,1
41,MDS-UPDRS #1: Finger Tapping,2017-07-07 10:00:07.797000-05:00,2017-07-07 10:00:19.664000-05:00,1,2
4,MDS-UPDRS #2: Hand Movements,2017-07-07 08:33:01.669000-05:00,2017-07-07 08:33:11.035000-05:00,2,1
42,MDS-UPDRS #2: Hand Movements,2017-07-07 10:00:25.635000-05:00,2017-07-07 10:00:34.817000-05:00,2,2
5,MDS-UPDRS #3: Pronation-Supination,2017-07-07 08:33:15.837000-05:00,2017-07-07 08:33:23.196000-05:00,3,1
43,MDS-UPDRS #3: Pronation-Supination,2017-07-07 10:00:42.179000-05:00,2017-07-07 10:00:50.002000-05:00,3,2


In [ ]:
def sort_sensor_data(path):

    files = os.listdir(path)
    
    for entries in files:
        if entries == 'annotations.csv':
            